<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')
// https://github.com/kmahelona/ipython_notebook_goodies

<IPython.core.display.Javascript object>

# Imports

In [2]:
%reload_ext autoreload
%autoreload 2

import TrackingAnalysis as TA
#import TrackingFigures as TF
import Utils

import collections
import numpy as np
import matplotlib.pyplot as plt
import functools
import pymongo
from pprint import pprint
import itertools

import tldextract
from IPy import IP
import csv
import time
import json
import operator
import copy
import re
import datetime
from requests.cookies import RequestsCookieJar
import cookies
from http.cookiejar import Cookie
import http.cookies
from urllib.parse import urlparse
import urllib.request
from tabulate import tabulate
from cookies import Cookie as ThirdPartyCookie
from collections import defaultdict


import seaborn
seaborn.set_style('whitegrid')

%matplotlib inline

print("Import success!")

Import success!


In [3]:
mongoHost = 'localhost'
mongoPort = 27017 #this is the default mongo port

In [4]:
# Graph parameters
xtickSize = 14
ytickSize = 14
ylabelSize = 18
xlabelSize = 18
titleSize = 22
legendSize = 14

In [5]:
usenixRunNames = {
    '1996': 'run:1/30/2016, 3:58:09 PM',
    '1997': 'run:1/30/2016, 4:59:48 PM',
    '1998': 'run:1/30/2016, 7:57:29 PM',
    '1999': 'run:2/1/2016, 2:57:51 PM',
    '2000': 'run:2/8/2016, 2:51:29 PM',
    '2001': 'run:2/7/2016, 10:57:46 PM',
    '2002': 'run:2/7/2016, 4:38:06 PM',
    '2003': 'run:1/25/2016, 7:06:03 PM',
    '2004': 'run:1/27/2016, 7:28:18 PM',
    '2005': 'run:1/29/2016, 6:20:15 PM',
    '2006': 'run:2/8/2016, 10:33:16 AM',
    '2007': 'run:1/27/2016, 3:45:28 PM',
    '2008': 'run:1/28/2016, 8:42:25 PM',
    '2009': 'run:1/26/2016, 5:10:40 PM',
    '2010': 'run:1/23/2016, 7:21:29 AM',
    '2011': 'run:1/22/2016, 8:53:47 PM',
    '2012': 'run:1/22/2016, 10:23:58 AM',
    '2013': 'run:1/21/2016, 11:51:32 PM',
    '2014': 'run:1/21/2016, 1:21:36 PM',
    '2015': 'run:2/8/2016, 11:07:13 PM',
    '2016': 'run:2/8/2016, 7:34:56 PM',
}
runNames = usenixRunNames

# Load the data

In [6]:
@Utils.timefunc
def reviveEngine(filename):
    import pickle
    with open(filename, 'rb') as f:
        return pickle.load(f)

: 

In [ ]:
def reviveWaybackEngines():
    import pickle
    import time
    
    engines = {}
    for year in range(1996, 2017):
        year = str(year)
        engines[year] = reviveEngine('/app/Data/waybackCollectedData/usenix%s.pickle' % year)  #set this to be your pickle location
        engines[year].year = year
    return engines
engines = reviveWaybackEngines()

reviveEngine took 0.27465367317199707 time
reviveEngine took 1.1509466171264648 time
reviveEngine took 1.3757250308990479 time
reviveEngine took 1.3781156539916992 time
reviveEngine took 2.009129524230957 time
reviveEngine took 2.3830626010894775 time
reviveEngine took 2.7427568435668945 time
reviveEngine took 7.810538053512573 time
reviveEngine took 4.985041618347168 time
reviveEngine took 4.34147834777832 time
reviveEngine took 6.900998592376709 time


In [ ]:
liveEngines = {
#     'Live2015Sep': reviveEngine('enginePickles/Live2015Sep.pickle'),
#     'Live2016Feb': reviveEngine('enginePickles/Live2016Feb.pickle'),
}    

# Graphs from the USENIX Paper

## How Well Are Trackers from 2016 Archived? (Figure 3)

In [ ]:
@functools.lru_cache(maxsize=1)
def colorsOfLiveTrackers(liveEngine, wbEngine):
    def trackersByPopularity(engine, typeFilter):
        instances = collections.Counter()
        for site in engine.getAllSitesWithTrackers():
            for tracker, types in engine.sitesToTrackersToTypes[site].items():
                if set(typeFilter) & types:  # Non-empty intersection of filter and types
                    instances[tracker] += 1
        return instances
    sortedTrackers = []
    sortedColors = []
    greens = TA.getRequestButNoCookies(liveEngine, wbEngine)
    reds = TA.getNoEvidence(liveEngine, wbEngine)
    smileys = TA.getSmileys(liveEngine, wbEngine)
    onlyInWB = TA.getOnlyInWayback(liveEngine, wbEngine)
    for tracker, instances in trackersByPopularity(liveEngine, 'BE').most_common():
        sortedTrackers.append(tracker)
        if tracker in smileys:
            sortedColors.append('Blue')
        elif tracker in greens:
            sortedColors.append('Green')
        elif tracker in reds:
            sortedColors.append('Red')
        elif tracker in onlyInWB:
            sortedColors.append('Purple')
        else:
            print('Confused about %s' % tracker)
            sortedColors.append('Confused')
            
    return sortedTrackers, sortedColors

def plotColorsLiveTrackers(liveEngine, wbEngine):


    colorToNumber = {
        'Blue': 0,
        'Green': 1,
        'Red': 2,
        'Confused': 3
    }
    sortedTrackers, sortedColors = colorsOfLiveTrackers(liveEngine, wbEngine)
    sortedColorNumbers = [colorToNumber[color] for color in sortedColors]
    
    # Try filtering for insignificance
    nonInsignificantTrackers = []
    nonInsignificantColors = []
    for (tracker, color) in zip(sortedTrackers, sortedColors):
        sitesForTrackerLive = liveEngine.getNumSitesWithTracker(tracker)
        sitesForTrackerWB = wbEngine.getNumSitesWithTracker(tracker)
        if sitesForTrackerLive != 1:
            nonInsignificantTrackers.append(tracker)
            nonInsignificantColors.append(color)
    
    def plotColors(colors, ax):
    
        fractionBlue = colors.count('Blue') / len(colors)
        fractionGreen = colors.count('Green') / len(colors)
        fractionRed = colors.count('Red') / len(colors)
    #     fractionPurple = colors.count('Purple') / len(colors)
        fractionConfused = colors.count('Confused') / len(colors)
        data = [fractionBlue, fractionGreen, fractionRed, fractionConfused]

        print(sum(data[0:3]))
        indices = np.arange(len(data))
        width = 1
        bars = ax.bar(indices, data, width)


        def autolabel(rects):
            for rect in rects:
                height = rect.get_height()
#                 print(height)
#                 print(rect.get_y())
                plt.text(rect.get_x()+rect.get_width()/2., rect.get_y()+1.05*height, '%d%%'%float(100*height),
                        ha='center', va='bottom', fontsize=14, fontweight='bold')


        bars[3].set_color('#a6611a')
        bars[2].set_color('#dfc27d')
        bars[1].set_color('#80cdc1')

        autolabel(bars)
    #     bars[3].set_color('purple')
        bars[0].set_color('#018571') 
        ax.set_xticks(indices+width/2)
        ax.set_xticklabels(['Live Tracker Also\nConfirmed In\nArchive', 
                            'Archive Sees\nTracker Only As\nThird Party', 
                            'No Third Party\nRequests For\nTracker', 
                            'Other'], ha='center', fontsize=TF.xtickSize, fontweight='bold')
        ax.set_title('How Well Are Trackers From 2016 Archived?', fontsize=TF.titleSize, fontweight='bold')
        ax.set_ylabel('Fraction of 2016\nLive Trackers', fontsize=TF.ylabelSize, fontweight='bold')
        ax.set_yticks(np.arange(0.0, 0.55, 0.05))
        ax.set_yticklabels(np.arange(0.0, 0.55, 0.05), fontsize=TF.ytickSize, fontweight='bold')

        ax.set_ylim(0, 0.55)
    #     indices = np.arange(len(sortedColors))
    #     ax.scatter(indices, sortedColorNumbers)


        #plt.savefig('figures/blueGreenRedFractionLiveWB.png', bbox_inches='tight')

    fig, axes = plt.subplots()
    fig.set_figwidth(10)
    plotColors(sortedColors, axes)
#     plotColors(nonInsignificantColors, axes[1])
    
    plt.tight_layout()
       
plotColorsLiveTrackers(liveEngines['Live2016'], engines['2016'])

## Causes of Archival Failures (Table 2)

In [ ]:
def robotsExclusions(wbEngine):
    counts, reqs, ratios = TA.robotsRequestsRatios(wbEngine)

    totalReqs = sum(reqs.values())
    totalDomains = len(set([TA.domainOf(e.get('request_url')) for e in wbEngine.processedRequestEvents]))
    totalURLs = len(set([e.get('request_url') for e in wbEngine.processedRequestEvents]))
    completeRobotsDomains = [d for d in ratios if ratios[d] == 100]

    
    robotExceptionURLs = [e['request_url'] for e in wbEngine.processedRequestEvents if TA.requestFailedDueToRobotAccessControlException(e)]
    print('%d requests blocked by robots' % len(robotExceptionURLs))
    print('%d distinct URLs blocked by robots' % len(set(robotExceptionURLs)))

    robotExceptionDomains = set([TA.domainOf(url) for url in robotExceptionURLs])
    print('%d distinct domains blocked by robots' % len(set(robotExceptionDomains)))
    print('%d/%d (%.1f%%) domains robots always' %
          (len(completeRobotsDomains), totalDomains, 100*len(completeRobotsDomains)/totalDomains))

def notArchived(wbEngine):
    counts, reqs, ratios = TA.requests404sRatios(wbEngine)

    totalReqs = sum(reqs.values())
    totalDomains = len(set([TA.domainOf(e.get('request_url')) for e in wbEngine.processedRequestEvents]))
    totalURLs = len(set([e.get('request_url') for e in wbEngine.processedRequestEvents]))
    
    exception404Requests = [e['request_url'] for e in wbEngine.processedRequestEvents if e.get('response_code') == 404]
    exception404DistinctURLs = set(exception404Requests)
    exception404DistinctDomains = set([TA.domainOf(url) for url in exception404DistinctURLs])
    complete404Domains = [d for d in ratios if ratios[d] == 100]
    
    print('%d/%d (%.1f%%) requests not archived (404 error)' % (len(exception404Requests), totalReqs, 100*len(exception404Requests)/totalReqs))
    print('%d/%d (%.1f%%) distinct URLs not archived (404 error) at least once' % 
          (len(exception404DistinctURLs), totalURLs, 100*len(exception404DistinctURLs)/totalURLs))
    
    print('%d/%d (%.1f%%) domains not archived (404 error) at least once' %
          (len(exception404DistinctDomains), totalDomains, 100*len(exception404DistinctDomains)/totalDomains))
    print('%d/%d (%.1f%%) domains not archived (404 error) always' %
          (len(complete404Domains), totalDomains, 100*len(complete404Domains)/totalDomains))


def waybackEscapes(wbEngine):
    counts, reqs, ratios = TA.bubbleRequestsRatios(wbEngine)
    totalReqsBubbled = sum(counts.values())
    totalReqs = sum(reqs.values())
    totalDomains = len(set([TA.domainOf(e.get('request_url')) for e in wbEngine.processedRequestEvents]))
    totalURLs = len(set([e.get('request_url') for e in wbEngine.processedRequestEvents]))
    distinctBubbledDomains = len([r for r in ratios.values() if r > 0])
    distinctBubbledURLs = len(set([e.get('request_url') for e in wbEngine.processedRequestEvents if e['blocked']]))
    completeBubbleDomains = [d for d in ratios if ratios[d] == 100]
    print('%d/%d (%.1f%%) requests escaped' % (totalReqsBubbled, totalReqs, 100*totalReqsBubbled/totalReqs))
    print('%d/%d (%.1f%%) distinct URLs escaped' % (distinctBubbledURLs, totalURLs, 100*distinctBubbledURLs/totalURLs))
    print('%d/%d (%.1f%%) domains escaped at least once' %
          (distinctBubbledDomains, totalDomains, 100*distinctBubbledDomains/totalDomains))    
    print('%d/%d (%.1f%%) domains escaoed always' %
          (len(completeBubbleDomains), totalDomains, 100*len(completeBubbleDomains)/totalDomains))

# NOTE: The events here are not included in the denominator!
def inconsistentTimestamps(wbEngine):
    counts, reqs, ratios = TA.inconsistentTimestampsRatios(wbEngine)
    totalReqsInconsistent = sum(counts.values())
    totalReqs = sum(reqs.values())
    totalDomains = len(set([TA.domainOf(e.get('request_url')) for e in wbEngine.processedRequestEvents]))
    totalURLs = len(set([e.get('request_url') for e in wbEngine.processedRequestEvents]))
    distinctInconsistentDomains = len([r for r in ratios.values() if r > 0])
    distinctInconsistentURLs = len(set([e.get('request_url') for e in wbEngine.processedRequestEvents if e['blocked']]))
    completeInconsistentDomains = [d for d in ratios if ratios[d] == 100]
    print('%d/%d (%.1f%%) requests had inconsistent timestamps' % (totalReqsInconsistent, totalReqs, 100*totalReqsInconsistent/totalReqs))
    print('%d/%d (%.1f%%) distinct URLs had inconsistent timestamps' % (distinctInconsistentURLs, totalURLs, 100*distinctInconsistentURLs/totalURLs))
    print('%d/%d (%.1f%%) domains had inconsistent timestamps at least once' %
          (distinctInconsistentDomains, totalDomains, 100*distinctInconsistentDomains/totalDomains))    
    print('%d/%d (%.1f%%) domains had inconsistent timestamps always' %
          (len(completeInconsistentDomains), totalDomains, 100*len(completeInconsistentDomains)/totalDomains))

In [ ]:
robotsExclusions(engines['2016'])
notArchived(engines['2016'])
waybackEscapes(engines['2016'])
inconsistentTimestamps(engines['2016'])

## Evolution of Tracker Types Over Time (Figure 4)

In [ ]:
def letterTrackingOverTime(engines, numSites, historicalDF, runNames):
    for y, eng in engines.items(): 
        siteList = TF.getOrderedTopNSites(mongo, runNames[y], numSites)
        setOfTrackers = defaultdict(set)
        for s in siteList:
            for ty in 'FEDCBA':
                tempSet = eng.getTrackersOfTypeOnSite(s, ty)
                setOfTrackers['total'] = setOfTrackers['total'].union(tempSet)
                setOfTrackers[ty] = setOfTrackers[ty].union(tempSet) # this function does not double-count B/C B/E

        for ty in 'ABCDEF':
            columnName = ty+str(numSites)
            historicalDF.at[int(y), columnName] = len(setOfTrackers[ty])
    
        totalCol = 'TotalTrackingDomains'+str(numSites)
        historicalDF.at[int(y), totalCol] = len(setOfTrackers['total'])
    historicalDF.sort(inplace=True)
    print(historicalDF)
    #writeHistoricalDF(historicalDF)  
    return historicalDF
  

def graphLetterTracking(engines, numSites, historicalDF, runNames):
    numSitesStr = str(numSites)
    historicalDF = letterTrackingOverTime(engines, numSites, historicalDF, runNames)
    statsList = ['A', 'B', 'C', 'D', 'E', 'F']
    typeNames = ['Analytics', 'Vanilla', 'Forced', 'Referred', 'Personal', 'Referred Analytics']
    colList = [stat+numSitesStr for stat in statsList]
    dfSlice = historicalDF[historicalDF[colList[1]] >= 0][colList]
    dfSlice.columns = typeNames

    #Make a graph
    sns.set_style("whitegrid")
    figDim = len(dfSlice)/1.75
    fig = plt.figure(figsize=(10, 6))
    ax = fig.gca()
    #plt.gca().tight_layout()
    #plt.gcf().subplots_adjust(bottom=0.50)
    #fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    my_colors = [(55,126,184),(77,175,74),(228,26,28),(152,78,163),(255,127,0),(166,86,40)]
    my_colors = [(r/255, g/255, b/255) for (r, g, b) in my_colors]
    my_dashes = [(None, None), [5,8], [5,3,2,3], [2,3], [5,2,5,2,5,10],
               [3,2,3,10], (None, None)]
    for ty, color, dash in zip(dfSlice.columns, my_colors, my_dashes):
        plt.plot(dfSlice.index, dfSlice[ty], color=color, dashes=dash, linewidth=5, label=ty)
    width=.8
    plt.bar(historicalDF.index-width/2,
              historicalDF['TotalTrackingDomains'+numSitesStr],
              width, color=(211/255, 211/255, 211/255), label='Total Tracker Domains')
    #plt.xticks(numpy.arange(1, len(historicalDF)+1)+1/2, historicalDF.index)
    ax.set_title('Trackers of Each Type In Dataset (Top {} Sites)'.format(numSitesStr), fontweight="bold", fontsize=titleSize)
    ax.set_ylabel('Trackers in Dataset', fontweight="bold", fontsize=ylabelSize)
    ax.set_xlabel('Year', fontweight="bold", fontsize=xlabelSize)
    ax.set_xticks(historicalDF.index)
    ax.set_xticklabels(historicalDF.index, rotation='45', fontweight="bold", fontsize=xtickSize)
    ax.set_yticklabels(ax.get_yticks().tolist(), fontweight="bold", fontsize=ytickSize)
    plt.xlim(dfSlice.index.values[0]-.5, dfSlice.index.values[-1]+.5)
    #ax.autoscale(axis='y', tight=True)
    ax.legend(loc=2, fontsize=legendSize+2)
    fig.set_tight_layout(True)
    fig.savefig('graphs/usenix/typesOverTime.png')
    
graphLetterTracking(wbEngines, 500, pd.DataFrame(), wbRunNamesForDataset)

## Complexity of Trackers (Table 4)

In [ ]:
def quantityOfTypesDistribution(engines, numSites, historicalDF, runNames, include3=False, live=False):
    typeList = ['A', 'B', 'C', 'D', 'E', 'F']
    colSuffix = 'Type_'+str(numSites)
    if include3:
        typeList.append('3')
        colSuffix += '_with3'

    multipleTypeCounter = defaultdict(Counter)
    for y, eng in engines.items():
        typesOfTracker = defaultdict(set)
        if not live:
            key = int(y)
        else:
            key = y
        siteList = TF.getOrderedTopNSites(mongo, runNames[y], numSites)
        # what we want is for each year how many trackers (on the subset of 450 sites had how many types)
        # so year -> tracker -> types_seen
        for s in siteList:
            for t in eng.getTrackersOnSite(s):
                tType = eng.getCombinedTypeOfTrackerOnSite(s, t).split(',')
                for ty in tType:
                    if ty in 'ABCDEF':
                        typesOfTracker[t].add(ty)
                
        totalTrackers = len(typesOfTracker)
        valueCounter = defaultdict(int)
        multipleTypeCounter[key] = Counter()
        for t, v in typesOfTracker.items():
            vLen = len(v)
            if 'E' in v or 'C' in v:
                vLen -= 1 # subtract out double-counted B's
            valueCounter[vLen] += 1
            if vLen > 1:
                multipleTypeCounter[key]["".join(v)] += 1
        for n in range(1, 1+len(typeList)):
            colName = str(n)+colSuffix
            valToInput = "0"
            if totalTrackers > 0:
                num = valueCounter[n]
            if num > 0:
                valToInput = "{:.2%} ({})".format(num/totalTrackers, num)
            historicalDF.at[key, colName] = valToInput
    historicalDF.sort(inplace=True)
    #print(historicalDF)
    #print(multipleTypeCounter)
    #tracker->TypeSet -> tracker->len(typeSet) -> len(typeSet)->len(trackerSet)
    # just make this a table

    return historicalDF, multipleTypeCounter

def tableQuantityOfTypes(engines, numSites, myDF, runNames, live=False):
    myDF, multipleTypes = quantityOfTypesDistribution(engines, numSites, myDF, runNames, include3=False, live=live)
    #print("Top {}".format(numSites))
    #for k in sorted(multipleTypes.keys()):
    #    if len(multipleTypes[k]) > 0:
    #        print("{}: {}".format(k, multipleTypes[k].most_common()))
    colNames = [str(n)+'Type_'+str(numSites) for n in range(1,5)] #no3
    if not live:
        keyList = [int(k) for k in runNames.keys()]
    else:
        keyList = runNames.keys()
    dfSlice = myDF[myDF.index.isin(keyList)]
    dfSlice = dfSlice.loc[:, colNames]
    #with open('graphs/usenix/typeDistribution.tex', 'w') as outfile:
    #    outfile.write(tabulate(dfSlice, headers=['Year']+[i[0:5] for i in colNames], tablefmt="latex"))
    print(tabulate(dfSlice, stralign="right", headers=['Year']+[i[0:5] for i in colNames]))
    
tableQuantityOfTypes(wbEngines, 500, pd.DataFrame(), wbRunNamesForDataset)

## Use of LocalStorage (Figure 6)

In [ ]:
def localStorageStackedBar():
    
    def getAllAPIEventsForYear(api, year):
        engine = engines[year]
        fpEvents = list(mongo.hammer.fingerprintApiEvent.find({'run_name': engine.run_name, 
                                                               'api_name': api}))
#         print('%d %s events in year %s' % (len(fpEvents), api, year))
        return fpEvents
    
    def isFirstPartyFPEvent(event):
        return event['frame']['parentFrameId'] == -1 and event['setting_script_domain'] == event['top_domain']
    def isThirdPartyFPEvent(event):
        return event['frame']['parentFrameId'] == -1 and event['setting_script_domain'] != event['top_domain']
    def isFramedFPEvent(event):
        return event['frame']['parentFrameId'] != -1
    
    def classifyFPEvent(event):
        if isFirstPartyFPEvent(event):
            return 'first party'
        elif isThirdPartyFPEvent(event):
            return 'third party'
        elif isFramedFPEvent(event):
            return 'framed'
        else:
            pprint(event)
            raise ValueError('confused about event')
            return 'confused'

        
    def getDataPerClassification(api):
        dataPerYear = []
        for year in range(1996, 2017):
            year = str(year)
       
            engine = engines[year]
            # Get all the localStorage FP events for each year.
            fpEvents = getAllAPIEventsForYear(api, year)
            # Filter out anachronisms.
    #         print('%d events' % len(fpEvents))
    #         print(engine.pastLimit, engine.futureLimit)
            fpEvents = [event for event in fpEvents if not TA.isAnachronism(event, engine.pastLimit, engine.futureLimit)]
    #         print('%d events after anachronism removal' % len(fpEvents))
            # Prepare to compute the below by calculating the script_setting_domain and top_domain from those URLs.        
            for event in fpEvents:
                event['setting_script_domain'] = TA.domainOf(TA.getArchivedURLFromWaybackURL(event['setting_script_url']))
                event['top_domain'] = TA.domainOf(TA.getArchivedURLFromWaybackURL(event['top_url']))
                event['frame_domain'] = TA.domainOf(TA.getArchivedURLFromWaybackURL(event['frame']['url']))
                event['classification'] = classifyFPEvent(event)


#             classifications = [e['classification'] for e in fpEvents]
#             tops = set([e['top_domain'] for e in fpEvents if e['classification'] == 'first party'])
#             thirds = set([e['setting_script_domain'] for e in fpEvents if e['classification'] == 'third party'])

#             realSites = set(TF.getOrderedTopNSites(mongo, engine.run_name, 500))
    #         print('%d/%d thirds are real top 500 sites' % (len(thirds & realSites), len(thirds)))
    #         print('%d/%d tops are real top 500 sites' % (len(tops & realSites), len(tops)))

    #         print(tops)
    #         allTopDomains = set([e['top_domain'] for e in fpEvents])
    #         allScriptDomains = set([e['setting_script_domain'] for e in fpEvents])
    #         print('tops', allTopDomains)
    #         print('scripts', allScriptDomains)

    #         print('%s: Intersection of all top and all script domains is size %d' % (year, len(allTopDomains & allScriptDomains)))

    #         print(tops)
    #         print(thirds)
    #         print('%s: Intersection of tops and thirds is size %d' % (year, len(tops & thirds)))
    #         print(tops & thirds)

            dataPerYear.append([
                len(set([e['top_domain'] for e in fpEvents if e['classification'] == 'first party'])), # first party
    #             classifications.count('first party'),
                len(set([e['setting_script_domain'] for e in fpEvents if e['classification'] == 'third party'])), # third party
    #             classifications.count('third party'),
    #             classifications.count('framed')
                len(set([e['frame_domain'] for e in fpEvents if e['classification'] == 'framed'])) # third party
            ])
        dataPerClassification = list(zip(*dataPerYear))
        dataPerClassification = ([np.array(row) for row in dataPerClassification])
        return dataPerClassification


    localStorageDataPerClassification = getDataPerClassification('window.localStorage')
#     doNotTrackDataPerClassification = getDataPerClassification('navigator.doNotTrack')
    print(localStorageDataPerClassification)
#     print(doNotTrackDataPerClassification)
    
#     dataPerClassification = doNotTrackDataPerClassification
#     bottoms = np.zeros(len(dataPerClassification[0]))
    
    fig, ax = plt.subplots()
    ax.plot(localStorageDataPerClassification[0], marker='o')
    ax.plot(localStorageDataPerClassification[1], color='orange', marker='o')
    ax.plot(localStorageDataPerClassification[2], color='red', marker='o')
    
#     ax[1].plot(doNotTrackDataPerClassification[0], marker='*')
#     ax[1].plot(doNotTrackDataPerClassification[1], color='orange', marker='*')
#     ax[1].plot(doNotTrackDataPerClassification[2], color='red', marker='*')
#     indices = range(len(dataPerClassification[0]))
#     ax.bar(indices, dataPerClassification[0])
#     bottoms += dataPerClassification[0]
#     ax.bar(indices, dataPerClassification[1], bottom=bottoms, color='orange')
#     bottoms += dataPerClassification[1]
#     ax.bar(indices, dataPerClassification[2], bottom=bottoms, color='red')
        
    ax.legend([
        'First party localStorage',
        'Third party localStorage',
        'Framed localStorage',
    ], loc='upper left', fontsize=12)
#     ax[1].legend([
#         'First party doNotTrack',
#         'Third party doNotTrack',
#         'Framed doNotTrack'
#     ], loc='upper left')
    
#     ax.set_xticklabels([])
    ax.set_xticklabels(range(1996, 2017, 5))

    ax.get_yaxis().set_label_coords(-0.075, 0.5)
    ax.set_ylabel('Domains', fontsize=TF.ylabelSize-2)
    ax.set_title('Use of localStorage', fontsize=TF.titleSize-4, fontweight='bold')
    ax.get_yaxis().set_label_coords(-0.025, 0.5)
    ax.get_yaxis().tick_right()

    fig.set_figheight(1.5)
    #fig.savefig('figures/localStorageAndDNT.png', bbox_inches='tight')
    
localStorageStackedBar()

## Distribution of Third Parties Requested Per Site (Figure 7)

In [ ]:
def boxThirdsPerYear(numSites, engines):
    years = []
    numReqs = []
    for y, eng in engines.items():
        siteList = TF.getOrderedTopNSites(mongo, wbRunNamesForDataset[y], numSites)
        for s in siteList:
            years.append(int(y))
            numReqs.append(eng.getNumTrackersOfTypeOnSite(s, '3'))
    
    #print(len(years))
    thirdsDF = pd.DataFrame({'year': years, 'numThirdPartyReqs': numReqs})
                        
   # print(df)
    thirdsDF = thirdsDF.sort_values('year')
    sns.set_style("whitegrid")
    fig = plt.figure(figsize=(12, 6))
    ax = fig.gca()
    ax = sns.boxplot(data=thirdsDF, x='year', y='numThirdPartyReqs', ax=ax, color="#deebf7", 
                     flierprops={'alpha':0.6,'markersize': 10, 'marker': '.'}, 
                    boxprops = dict(linewidth=3, color='black'), 
                    whiskerprops = dict(linewidth=3, color='black'),
                    capprops = dict(linewidth=3, color='black'),
                    medianprops = dict(linewidth=3, color='black'))
    plt.title('Third Parties Requested Per Site (Top {} Sites)'.format(numSites), fontweight="bold", fontsize=titleSize)
    plt.ylabel('Third Parties Requested per Site', fontweight="bold", fontsize=ylabelSize)
    plt.xlabel('Year', fontweight="bold", fontsize=xlabelSize)
    plt.xticks(fontweight="bold", fontsize=xtickSize, rotation='45', ha='right')
    plt.yticks(fontweight="bold", fontsize=ytickSize)
    #fig.savefig('graphs/usenix/boxplotThirds'+str(numSites)+'.png')
    plt.show()
    
boxThirdsPerYear(500, wbEngines)

## Distribution of Third-Party Requests (Figure 8)

In [ ]:
def thirdCDF(engines, numSites, historicalDF, runNames):
    # eventually want a number of trackers (0-max ever seen+1)-> % of domains per year 
    # (% of domains just means dividing by total # of domains)
    # year is still index. Columns for each number of trackers? Yikes.
    beforePercentsTable = defaultdict(list)
    maxNumDomainsOnSite = 0
    for y, eng in engines.items():
        siteList = TF.getOrderedTopNSites(mongo, runNames[y], numSites)
        thirdReqs = Counter()
        for s in siteList:
            thirdReqs[s] = eng.getNumTrackersOfTypeOnSite(s, '3')
        # now I have a counter with the number of thirdReqs for each site that year
        # counter most_common gives me the largest count
        key = int(y)
        totalSites = len(siteList)
        mostCommonCount = thirdReqs.most_common(1)[0][1]
        maxNumDomainsOnSite = max(maxNumDomainsOnSite, mostCommonCount)
        #print(mostCommonCount)
        howManyDomainsWithThirdCount = defaultdict(int)
        for (s, numThirdReqs) in thirdReqs.most_common():
            howManyDomainsWithThirdCount[numThirdReqs] += 1
        #print("{} {}".format(key, howManyDomainsWithThirdCount))
        # so now what I have is the count of how many domains have each tracker number
        # to get how many have at least each tracker number add the sum of everything greater
        howManyDomainsWithAtLeastThirdCount = defaultdict(int)
        for n in reversed(range(1+mostCommonCount)):
            howManyDomainsWithAtLeastThirdCount[n] = howManyDomainsWithThirdCount[n] + howManyDomainsWithAtLeastThirdCount[n+1]
        beforePercentsTable[key] = [(n, v) for n, v in howManyDomainsWithAtLeastThirdCount.items()]
        #historicalDF.at[key, "AtLeast0Capable_"+str(numSites)] = 100.0
        for thirdCount, numDomains in howManyDomainsWithAtLeastThirdCount.items():
            historicalDF.at[key, "AtLeast{}Capable_{}".format(thirdCount, numSites)] = 100*numDomains/totalSites
        historicalDF.at[key, "AtLeast0Capable_"+str(numSites)] = 100.0
        historicalDF.at[key, "AtLeast{}Capable_{}".format(mostCommonCount+1, numSites)] = 0.0
    beforePercentsTableItems = beforePercentsTable.items()
    rawDF = pd.DataFrame({'Year': [k for k,_ in beforePercentsTableItems],
                        'NumTracker, HowManyDomainsHaveAtLeastNumTrackerTrackers': [v for _,v in beforePercentsTableItems]
                       })
    #print(rawDF)
    #rawDF.to_csv('graphs/rawReverseCDF_top{}.csv'.format(numSites))
    historicalDF.sort(inplace=True)
    #writeHistoricalDF(historicalDF)
    return historicalDF, maxNumDomainsOnSite+1

def graphThirdCDF(engines, numSites, historicalDF, runNames, showEven=False):
    historicalDF, maxColNumber = thirdCDF(engines, numSites, historicalDF, runNames)
    print("Max col number: {}".format(maxColNumber))
    columnsToSlice = ["AtLeast{}Capable_{}".format(i, numSites) for i in range(1+maxColNumber)]
    dfSlice = historicalDF[historicalDF[columnsToSlice[0]] > 0][columnsToSlice]
    dfSlice.columns = range(1+maxColNumber)
    #print(dfSlice.columns)
    sns.set_style("whitegrid")
    #figDim = len(dfSlice)/1.5
    fig = plt.figure(figsize=(11, 5))
    ax = fig.gca()
    plt.title('Distribution of Third-Party Requests (Top {} Sites)'.format(numSites), fontweight="bold", fontsize=titleSize)
    plt.ylabel('Percentage of Sites', fontweight="bold", fontsize=ylabelSize)
    plt.xlabel('Minimum Number of Third-Parties', fontweight="bold", fontsize=xlabelSize)
    if showEven:
        howManyYears = len([y for y in dfSlice.index.values if y % 2 != 1])
    else:
        howManyYears = len(dfSlice.index.values)
    my_colors = sns.cubehelix_palette(howManyYears, light=.7, rot=-1.2, reverse=True)
    for year, i in zip(dfSlice.index.values, range(2*len(my_colors))):
        if not showEven:
            dfSlice2 = dfSlice.loc[year, :]
            dfSlice2.plot(kind='line', color=my_colors[i], label=year, linewidth=4)
        elif year % 2 != 1:
            dfSlice2 = dfSlice.loc[year, :]
            dfSlice2.plot(kind='line', color=my_colors[int(math.floor(i/2))],
                        label=year, linewidth=4)
    plt.xticks(fontweight="bold", fontsize=xtickSize) #rotation='45', ha='right')
    plt.yticks(fontweight="bold", fontsize=ytickSize)  
    ax.legend(loc='upper right', fontsize=legendSize)
    plt.minorticks_on()
    plt.grid(b=True, which='minor')
    fig.set_tight_layout(True)
    #fig.savefig('graphs/usenix/thirdPartiesCDF'+str(numSites)+'.png')
    plt.show()

graphThirdCDF(wbEngines, 500, pd.DataFrame(), wbRunNamesForDataset, showEven=True)

## Coverage Evolution Over Time (Figure 9)

In [ ]:
# Utility functions for coverage

def percentSitesContaining(sortedEngineYears, sortedEngines, typeFilter):
    def trackersByPopularity(engine, typeFilter):
        instances = collections.defaultdict(set)
        for site in engine.getAllSitesWithTrackers():
            for tracker, types in engine.sitesToTrackersToTypes[site].items():
                if set(typeFilter) & types:  # Non-empty intersection of filter and types
                    instances[tracker].add(site)
        return instances
    
    def mostCommon(dictOfSets, N):        
        return sorted(((tracker, sites) for tracker, sites in dictOfSets.items()), reverse=True, key=lambda x: len(x[1]))[:N]

    def ratiosForEngine(engine):
        instances = trackersByPopularity(engine, typeFilter)
#         total = len(engine.getAllSitesWithTrackers())
        total = len(engine.getAllSites())
        #print('There are %d sites with trackers' % total)
        try:
            top1Sites = set.union(*[t[1] for t in mostCommon(instances, 1)])
            top1Count = len(top1Sites)
            top5Sites = set.union(*[t[1] for t in mostCommon(instances, 5)])
            top5Count = len(top5Sites)
            top10Sites = set.union(*[t[1] for t in mostCommon(instances, 10)])
            top10Count = len(top10Sites)
            top20Sites = set.union(*[t[1] for t in mostCommon(instances, 20)])
            top20Count = len(top20Sites)
            
#             print('Top 20 in %s: %s' % (engine.year, top20Sites))
       
            top1Ratio = 100*top1Count/total
            top5Ratio = 100*top5Count/total
            top10Ratio = 100*top10Count/total
            top20Ratio = 100*top20Count/total
        except: 
            print('Year %s has total 0 tracking' % year)
            top1Ratio = 0
            top5Ratio = 0
            top10Ratio = 0
            top20Ratio = 0
        
        return top1Ratio, top5Ratio, top10Ratio, top20Ratio

    data = []
    for year, engine in zip(sortedEngineYears, sortedEngines):
        top1Ratio, top5Ratio, top10Ratio, top20Ratio = ratiosForEngine(engine)
        data.append([year, top1Ratio, top5Ratio, top10Ratio, top20Ratio])
    return data

In [ ]:
def getColors():
    colors = [
        '#fbb4b9',
        '#f768a1',
        '#c51b8a',
        '#7a0177'
    ]
    for color in colors:
        yield color
        
def linePlot(data, ax, typeFilter, legendLocation, yLabel=True):
    import matplotlib.ticker as mtick
    years, top1s, top5s, top10s, top20s = zip(*data)
    topData = [top20s, top10s, top5s, top1s]
    for d, color in zip(topData, getColors()):
        ax.plot(d, color=color)
#     ax.bar(range(len(years), len(years)+4), 
#            [live1Ratio, live5Ratio, live10Ratio, live20Ratio],
#           color='yellow')

    ax.legend( #(line1, line5, line10, line20),
                ('Top 20 Trackers', 'Top 10 Trackers', 'Top 5 Trackers', 'Top 1 Tracker'), 
                loc=legendLocation, fontsize=legendSize)
# Utility functions for coverage

def percentSitesContaining(sortedEngineYears, sortedEngines, typeFilter):
    def trackersByPopularity(engine, typeFilter):
        instances = collections.defaultdict(set)
        for site in engine.getAllSitesWithTrackers():
            for tracker, types in engine.sitesToTrackersToTypes[site].items():
                if set(typeFilter) & types:  # Non-empty intersection of filter and types
                    instances[tracker].add(site)
        return instances
    
    def mostCommon(dictOfSets, N):        
        return sorted(((tracker, sites) for tracker, sites in dictOfSets.items()), reverse=True, key=lambda x: len(x[1]))[:N]

    def ratiosForEngine(engine):
        instances = trackersByPopularity(engine, typeFilter)
#         total = len(engine.getAllSitesWithTrackers())
        total = len(engine.getAllSites())
        #print('There are %d sites with trackers' % total)
        try:
            top1Sites = set.union(*[t[1] for t in mostCommon(instances, 1)])
            top1Count = len(top1Sites)
            top5Sites = set.union(*[t[1] for t in mostCommon(instances, 5)])
            top5Count = len(top5Sites)
            top10Sites = set.union(*[t[1] for t i# Utility functions for coverage

def percentSitesContaining(sortedEngineYears, sortedEngines, typeFilter):
    def trackersByPopularity(engine, typeFilter):
        instances = collections.defaultdict(set)
        for site in engine.getAllSitesWithTrackers():
            for tracker, types in engine.sitesToTrackersToTypes[site].items():
                if set(typeFilter) & types:  # Non-empty intersection of filter and types
                    instances[tracker].add(site)
        return instances
    
    def mostCommon(dictOfSets, N):        
        return sorted(((tracker, sites) for tracker, sites in dictOfSets.items()), reverse=True, key=lambda x: len(x[1]))[:N]

    def ratiosForEngine(engine):
        instances = trackersByPopularity(engine, typeFilter)
#         total = len(engine.getAllSitesWithTrackers())
        total = len(engine.getAllSites())
        #print('There are %d sites with trackers' % total)
        try:
            top1Sites = set.union(*[t[1] for t in mostCommon(instances, 1)])
            top1Count = len(top1Sites)
            top5Sites = set.union(*[t[1] for t in mostCommon(instances, 5)])
            top5Count = len(top5Sites)
            top10Sites = set.union(*[t[1] for t in mostCommon(instances, 10)])
            top10Count = len(top10Sites)
            top20Sites = set.union(*[t[1] for t in mostCommon(instances, 20)])
            top20Count = len(top20Sites)
            
#             print('Top 20 in %s: %s' % (engine.year, top20Sites))
       
            top1Ratio = 100*top1Count/total
            top5Ratio = 100*top5Count/total
            top10Ratio = 100*top10Count/total
            top20Ratio = 100*top20Count/total
        except: 
            print('Year %s has total 0 tracking' % year)
            top1Ratio = 0
            top5Ratio = 0
            top10Ratio = 0
            top20Ratio = 0
        
        return top1Ratio, top5Ratio, top10Ratio, top20Ratio

    data = []
    for year, engine in zip(sortedEngineYears, sortedEngines):
        top1Ratio, top5Ratio, top10Ratio, top20Ratio = ratiosForEngine(engine)
        data.append([year, top1Ratio, top5Ratio, top10Ratio, top20Ratio])
    return datan mostCommon(instances, 10)])
            top10Count = len(top10Sites)
            top20Sites = set.union(*[t[1] for t in mostCommon(instances, 20)])
            top20Count = len(top20Sites)
            
#             print('Top 20 in %s: %s' % (engine.year, top20Sites))
       
            top1Ratio = 100*top1Count/total
            top5Ratio = 100*top5Count/total
            top10Ratio = 100*top10Count/total
            top20Ratio = 100*top20Count/total
        except: 
            print('Year %s has total 0 tracking' % year)
            top1Ratio = 0
            top5Ratio = 0
            top10Ratio = 0
            top20Ratio = 0
        
        return top1Ratio, top5Ratio, top10Ratio, top20Ratio

    data = []
    for year, engine in zip(sortedEngineYears, sortedEngines):
        top1Ratio, top5Ratio, top10Ratio, top20Ratio = ratiosForEngine(engine)
        data.append([year, top1Ratio, top5Ratio, top10Ratio, top20Ratio])
    return data
    ax.set_title('%s Coverage' % typeFilter, y=1.04, fontsize=titleSize, fontweight='bold')
    ax.set_xlabel('Years', fontsize=xlabelSize, fontweight='bold')
    if yLabel:
        ax.set_ylabel('% Sites', fontsize=ylabelSize, fontweight='bold')
    ax.set_ylim(0, 100)
    ax.set_xticks(range(len(years)))
    ax.set_xticklabels(years, rotation=45, ha='right', fontsize=xtickSize, fontweight='bold')
    ax.set_yticklabels(ax.get_yticks().tolist(), fontsize=ytickSize, fontweight='bold')
#         ax.set_ylim(0, .7)

    for label in ax.xaxis.get_ticklabels()[1::2]:
        label.set_visible(False)

    fmt = '%.0f%%' # Format you want the ticks, e.g. '40%'
    yticks = mtick.FormatStrFormatter(fmt)
    ax.yaxis.set_major_formatter(yticks)#set_yticks(range(0, 40, 5))

def barPlot(data, ax):
    import matplotlib.ticker as mtick

    indices = np.arange(0, len(data)/2, 0.5)
    width = 0.35
    bars = ax.bar(indices, data, width)
    colors = getColors()

#     fmt = '%.0f%%' # Format you want the ticks, e.g. '40%'
#     yticks = mtick.FormatStrFormatter(fmt)
#     ax.yaxis.set_major_formatter(yticks)#set_yticks(range(0, 40, 5))

    for i, color in enumerate(reversed(list(colors))):
        bars[i].set_color(color)
    ax.set_title('2016 Tracker Coverage', y=1.04, fontsize=titleSize, fontweight='bold')
    ax.set_ylim(0, 100)
    ax.set_yticks(np.arange(0, 120, 20))
    ax.set_yticklabels(['%d%%' % n for n in np.arange(0, 120, 20)], fontsize=ytickSize, fontweight='bold')
    ax.set_xticks(indices + 0.5*width)
    ax.set_xticklabels(['Top 1', 'Top 5', 'Top 10', 'Top 20'], 
                       ha='center', fontsize=xtickSize, fontweight='bold')
    

def plot3Panels():
    # 2003 is the first year with >20 trackers. (Checked below)
    data3s = percentSitesContaining(TA.sortedEngineYears(engines, firstYear='1996'), 
                                    TA.sortedEngines(engines, firstYear='1996'), 
                                    '3')
#     dataBs = percentSitesContaining(sortedEngineYears(firstYear='1996'), sortedEngines(firstYear='1996'), 'B')
    dataAll = percentSitesContaining(TA.sortedEngineYears(engines, firstYear='1996'), 
                                     TA.sortedEngines(engines, firstYear='1996'), 
                                     'ABCDEF')
#     liveBs = percentSitesContaining(['Live'], [liveEngine], 'B')
    liveAll = percentSitesContaining(['Live'], [liveEngines['Live2016FebNSDI']], 'ABCDEF')

    fig, axes = plt.subplots(1,3)
    fig.set_tight_layout(True)
    fig.set_figwidth(14)
    fig.set_figheight(5)
    linePlot(data3s, axes[1], 'Third Party', 'upper left', yLabel=False)
#     linePlot(dataBs, axes[0], 'Vanilla', 'upper left', yLabel=True)
    linePlot(dataAll, axes[0], 'Tracker', 'upper left', yLabel=True)

    print(liveAll)
#     barPlot(liveBs[0][1:], axes[2])
    barPlot(liveAll[0][1:], axes[2])

    #plt.savefig('figures/coverage3Panel.png')
plot3Panels()

## Rise and Fall of Historical Champion Trackers (Figure 10)

In [ ]:
# Utility functions for champions

def sortedEngines(firstYear=None): 
    if firstYear:
        return [e[1] for e in sorted(engines.items()) if e[0] >= firstYear]
    else:
        return [e[1] for e in sorted(engines.items())]
def sortedEngineYears(firstYear=None):
    if firstYear:
        return [e[0] for e in sorted(engines.items()) if e[0] >= firstYear]
    else:
        return [e[0] for e in sorted(engines.items())]
def getMostPopularTrackersForEngine(engine, typeFilter, topN, year, numChampions=1, insignificance=1):
    mongo = pymongo.MongoClient(host=mongoHost, port=mongoPort)
    topNSites = set(TF.getOrderedTopNSites(mongo, runNames[year], topN))
    instances = trackersByPopularityWithSiteFilter(engine, typeFilter, siteFilter=topNSites)
    if len(instances) == 0:
        return None
    for nonTracker in ['.graphics', '.Graphics', '.pictures', '.pics', '.download', '.media']:
        del instances[nonTracker]
    champions = instances.most_common(numChampions)
    print(engine.year, instances.most_common(3))
#     print(year)
#     pprint.pprint(instances.most_common())
    
    trackersToReturn = [x[0] for x in champions if x[1] > insignificance] # Return just names not counts.
#     print('returning', trackersToReturn, champions)
    return trackersToReturn
#     return instances.most_common(1)[0][0]

def trackersByPopularityWithSiteFilter(engine, typeFilter, siteFilter=set()):
    instances = collections.Counter()
#     print('trackers by popularity with %d sites' % len(set(engine.getAllSitesWithTrackers() & siteFilter)))
    for site in engine.getAllSitesWithTrackers():
        if site not in siteFilter:
            continue
        for tracker, types in engine.sitesToTrackersToTypes[site].items():
            if set(typeFilter) & types:  # Non-empty intersection of filter and types
                instances[tracker] += 1
    return instances

def dedupeWithOrder(seq):
    seen = set()
    seen_add = seen.add
    return [ x for x in seq if not (x in seen or seen_add(x))]

@functools.lru_cache(maxsize=6)
def getChampionsWithYearlyCoverage(typeFilter, firstYear='1996', topN=500, numChampions=1):
    orderedChampions = [getMostPopularTrackersForEngine(engine, typeFilter, topN, year, numChampions=numChampions) 
                        for engine, year in zip(TA.sortedEngines(engines, firstYear=firstYear), TA.sortedEngineYears(engines, firstYear=firstYear))]
    pprint(orderedChampions)
    orderedChampions = [champ for sublist in orderedChampions for champ in sublist]
    if None in orderedChampions:
        orderedChampions.remove(None)
    orderedChampions = dedupeWithOrder(orderedChampions)
    print('orderedChampions', orderedChampions)
    champions = set(orderedChampions)
    champions.discard(None)
    championToYearlyCoverages = collections.defaultdict(list)
    mongo = pymongo.MongoClient(host=mongoHost, port=mongoPort)
    for year in TA.sortedEngineYears(engines, firstYear=firstYear):
        topNSites = set(TF.getOrderedTopNSites(mongo, runNames[year], topN))
#         print('%d topNSites where N is %d' % (len(topNSites), topN))
        instances = trackersByPopularityWithSiteFilter(engines[year], typeFilter, siteFilter=topNSites)
        totalSites = len(topNSites)
        for champ in champions:
            if champ in instances:
                championToYearlyCoverages[champ].append(instances[champ]/totalSites)
            else:
                championToYearlyCoverages[champ].append(0)
    return championToYearlyCoverages, orderedChampions

In [ ]:
# getOrderedTopNSites()
def careers(firstYear='1996', topN=500, numChampions=1, typeFilter='3B', ax=None):
    def colors():
#         someColors = ['#8dd3c7', '#ffffb3', '#bebada', '#fb8072', '#80b1d3', 
#                       '#fdb462', '#b3de69', '#fccde5', '#d9d9d9', '#bc80bd', '#ccebc5']:
#         someColors = ["#9b59b6", "#3498db", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71"]
        someColors = ["windows blue", "amber", "greyish", "faded green", "pink", "charcoal"]

#         palette = seaborn.color_palette("Set", 16)
        palette = seaborn.xkcd_palette(someColors)
#         palette[5] = '#663399'
#         for color in palette:
#             yield color
        while True:
            for color in ['#000000','#1f78b4','#b2df8a','#e31a1c','#fb9a99','#33a02c',
                          '#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#b15928','#bbbb99',]:
                yield color

    championToYearlyCoverage, orderedChampions = getChampionsWithYearlyCoverage(typeFilter, firstYear=firstYear, 
                                                                                topN=topN, numChampions=numChampions)
    pprint(orderedChampions)
    excludeList = [
        'riddler.com', 'ciec.org', 'wired.com', 'hongkong.com', 'akamai.net', 'netguide.com', 'foxworld.com', # manual
        'count$wow.net', '.graphics',  # not a URL?
        'googleapis.com'  # Only a F> tracker (it sends cookies to enable an F)
    ]

    championToYearlyCoverage = {k: v for k, v in championToYearlyCoverage.items() if k not in excludeList and k[0] != '.'}

    orderedChampions = [c for c in orderedChampions if c not in excludeList and c[0] != '.']
    
    
    
    indices = np.arange(len(TA.sortedEngineYears(engines, firstYear=firstYear)))
    legendArg = []
    print(orderedChampions)
    for champ, color in zip(orderedChampions, colors()):
        coverages = championToYearlyCoverage[champ]
        if champ == 'google-analytics.com':
            line = ax.plot(coverages, color=color, linewidth=5, marker='*', markersize=18)
        else:
            line = ax.plot(coverages, color=color, linewidth=5)
        legendArg.append(champ)

    ax.legend(legendArg, loc=2, fontsize=legendSize+4)
    ax.set_xlabel('Years', fontsize=xlabelSize, fontweight='bold')
    ax.set_ylabel('Coverage (of Top 500)', fontsize=ylabelSize, fontweight='bold')
    if typeFilter == 'ABCDEF':
        trackerName = 'Confirmed Tracker'
    elif typeFilter =='3ABCDEF':
        trackerName = 'Tracking-Capable or Confirmed'
    ax.set_title('Rise And Fall of Historical Champion Trackers', fontsize=titleSize, fontweight='bold')
    ax.set_xticks(indices)
    ax.set_xticklabels(TA.sortedEngineYears(engines, firstYear=firstYear), 
                       rotation=45, ha='right', fontsize=xtickSize, fontweight='bold')
    ax.set_yticks(np.arange(0, 0.5, 0.05))
    ax.set_yticklabels(np.arange(0, 0.5, 0.05), fontsize=xtickSize, fontweight='bold')

    #plt.savefig('figures/championCoverage.png', bbox_inches='tight')

def bunchaCareers():
    fig, axes = plt.subplots(2, 2, figsize=(16,12))
    careers(firstYear='1996', topN=100, numChampions=1, typeFilter='ABCDEF', ax=axes[0][0])
    careers(firstYear='1996', topN=100, numChampions=2, typeFilter='3ABCDEF', ax=axes[0][1])
    careers(firstYear='1996', topN=450, numChampions=1, typeFilter='ABCDEF', ax=axes[1][0])
    careers(firstYear='1996', topN=450, numChampions=2, typeFilter='3ABCDEF', ax=axes[1][1])

def justForPaper():
    fig, ax = plt.subplots(figsize=(14, 6))
    careers(firstYear='1996', topN=500, numChampions=2, typeFilter='3ABCDEF', ax=ax)

# bunchaCareers()
justForPaper()

## Referrer Complexity (Figure 11)

In [ ]:
def graphComplexityTable(graphs):
    from tabulate import tabulate
    data = []

    meanDegrees = []
    medianDegrees =[]
    withZeroInDegrees = []
    withZeroOutDegrees = []
    maxInDegrees = []
    maxOutDegrees = []

    for year, graph in graphs:
        inDegrees = [graph.inc_degree(n) for n in graph.node_list()]
        outDegrees = [graph.out_degree(n) for n in graph.node_list()]

        meanDegrees.append(np.mean(inDegrees))
        medianDegrees.append(np.median(inDegrees))
        withZeroInDegrees.append(100 * inDegrees.count(0)/len(inDegrees))
        withZeroOutDegrees.append(100 * outDegrees.count(0)/len(outDegrees))
        maxInDegrees.append(max(inDegrees))
        maxOutDegrees.append(max(outDegrees))
        
        data.append([
                year,
                '%.2f' % (np.mean(inDegrees)), 
                '%.1f%%' % (100 * inDegrees.count(0)/len(inDegrees)),
                '%.1f%%' % (100 * outDegrees.count(0)/len(outDegrees)),
                max(inDegrees),
                max(outDegrees),
        ])
    #print(tabulate(data, tablefmt='latex'))
    
    fig, ax = plt.subplots(3, 1)
    fig.set_figheight(6)
    ax[0].plot(meanDegrees, color='black')
    ax[0].plot(medianDegrees, marker='o', linestyle='None', color='black')
    ax[0].set_ylabel('References', fontsize=TF.ylabelSize-2)
    ax[0].legend(('Mean references', 'Median references',), loc='upper left', fontsize=14)
    ax[0].set_xticks(range(0, 21, 2))
    ax[0].set_xticklabels([])
    ax[0].set_yticklabels([str(x) for x in np.arange(0, 3, 0.5)], fontsize=TF.ytickSize)
    
    ax[1].plot(withZeroInDegrees, linestyle='--', color='black')
    ax[1].plot(withZeroOutDegrees, color='black')
    ax[1].set_ylabel('Fraction...', fontsize=TF.ylabelSize-2)
    ax[1].legend(('... never referred to', '... that refers nothing'), loc='lower left', fontsize=13)
    ax[1].set_ylim(0,100)
    ax[1].set_xticks(range(0, 21, 2))
    ax[1].set_yticklabels(np.arange(0, 1.2, 0.2), fontsize=TF.ytickSize)
#     ax[1].get_yaxis().tick_right()

    ax[1].set_xticklabels([])
    
    ax[2].plot(maxInDegrees, linestyle='--', color='black')
    ax[2].plot(maxOutDegrees, color='black')
    ax[2].set_ylabel('References', fontsize=TF.ylabelSize-2)
    ax[2].legend(('Max times referred', 'Max 3rd parties\nreferred'), loc='upper left', fontsize=14)
    ax[2].set_ylim(0,200)
    ax[2].set_xticks(range(0, 21, 4))
    ax[2].set_xticklabels([str(year) for year in range(1996, 2017, 4)], fontsize=TF.xtickSize)
    ax[2].set_yticklabels([str(x) for x in range(0, 250, 50)], fontsize=TF.ytickSize)
    
    for a in ax:
        a.get_yaxis().set_label_coords(-0.125, 0.5)

    fig.suptitle('Referer Complexity', fontsize=TF.titleSize, fontweight='bold')
    fig.subplots_adjust(bottom=-0.15)
    fig.set_figheight(4.5)
    #fig.savefig('figures/whaleComplexity.png', bbox_inches='tight')
    
def averageInDegree(graph):
    inDegrees = [graph.inc_degreee(n) for n in graph.node_list()]
    
    mean = np.mean(inDegrees)
    median = np.median(inDegrees)
    print('mean: %f' % mean)
    print('median: %f' % median)
    print('max: %d' % max(inDegrees))
    print('0s: %d (%.2f%%)' % (inDegrees.count(0), 100*inDegrees.count(0)/len(inDegrees)))

graphComplexityTable([( str(year), graphs[str(year)] ) for year in range(1996, 2017)])